# 1  데이터 수집 1 – 한 페이지 크롤링

다나와 사이트에서 제품별로 용량, 크기 같은 조건을 설정해서 검색하여

무선청소기 제품의 가격과 정보를 수집한다.

[도서]  [데이터실무분석 with파이썬 예제](https://wikibook.co.kr/playwithdata/)

In [9]:
### 6.1.1 패키지 설치
from bs4 import BeautifulSoup 
import time
import pandas as pd

In [11]:
# ------------------- [ 2024.12 ] 설치없이 실행 가능

# selenium4
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

# [2024.05] Jupyter notebook (anaconda)에서 오류 발생
# python version : 3.11.7
# selenium version :  4.21.0
#from webdriver_manager.chrome import ChromeDriverManager ==> [에러 발생]

options = Options()
# 브라우저 꺼짐 방지 코드
options.add_experimental_option("detach", True)
# 불필요한 에러 메세지 제거 코드
# options.add_experimental_option("excludeSwitched",["enable-logging"]);
# service = Service(ChromeDriverManager().install()) # [ 패키지부터 에러발생 ]


driver = webdriver.Chrome(options=options)
driver.implicitly_wait(3)


### 6.1.2 다나와 검색 페이지 접속

In [5]:
# 예제 6-1 selenium으로 다나와 검색 결과 URL에 접속
from selenium import webdriver


# 다나와 사이트에서 실제 '무선청소기' 검색 후 url 확인
url = "http://search.danawa.com/dsearch.php?query=가습기&tab=main" 
driver.get(url)

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ff57cda5+78885]
	GetHandleVerifier [0x0x7ff6ff57ce00+78976]
	(No symbol) [0x0x7ff6ff339bca]
	(No symbol) [0x0x7ff6ff3259b5]
	(No symbol) [0x0x7ff6ff34a9ca]
	(No symbol) [0x0x7ff6ff3c05e5]
	(No symbol) [0x0x7ff6ff3e0b42]
	(No symbol) [0x0x7ff6ff3b8963]
	(No symbol) [0x0x7ff6ff3816b1]
	(No symbol) [0x0x7ff6ff382443]
	GetHandleVerifier [0x0x7ff6ff854eed+3061101]
	GetHandleVerifier [0x0x7ff6ff84f33d+3037629]
	GetHandleVerifier [0x0x7ff6ff86e592+3165202]
	GetHandleVerifier [0x0x7ff6ff59730e+186766]
	GetHandleVerifier [0x0x7ff6ff59eb3f+217535]
	GetHandleVerifier [0x0x7ff6ff5859b4+114740]
	GetHandleVerifier [0x0x7ff6ff585b69+115177]
	GetHandleVerifier [0x0x7ff6ff56c368+10728]
	BaseThreadInitThunk [0x0x7ffc1be2e8d7+23]
	RtlUserThreadStart [0x0x7ffc1c9bc34c+44]

### 6.1.3 다나와 검색 웹 페이지에서 상품 정보 가져오기

크롬에서 F12(개발자모드)를 보면서 확인합시다

In [4]:
from bs4 import BeautifulSoup

#예제 6-2 웹 페이지의 HTML 정보 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# 예제 6-3 1페이지에 대한 무선청소기 정보 가져오기 1
prod_items = soup.select('li.prod_item')
len(prod_items)

30

In [6]:
# 예제 6-4 1페이지에 대한 무선청소기 정보 가져오기 2 - 상위 태그를 추가해서 확인
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

30

In [7]:
# 예제 6-5 1페이지에 대한 무선청소기 정보 가져오기 3 - 상위 태그를 추가해서 확인
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

30

In [8]:
# 예제 6-6 상품명 정보 가져오기
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

조지루시 EE-DVH35K


In [9]:
# 예제 6-7 스펙 목록 정보 가져오기
spec_list = prod_items[0].select('div.spec_list')[0].text.strip()
print(spec_list)

가습기/가열식/수조, 분무구세척/[가습] 수조용량: 4.0L/연속사용: 10시간/최대분무량: 370cc/습도조절/습도표시/분무량조절/[편의] 상부급수식/타이머/[규격] 소비전력: 680W/크기(가로x세로x깊이): 240x365x275mm


In [10]:
prod_items[0]

prod_items[0].select('li p.price_sect > a > strong')

[<strong>283,880</strong>]

In [11]:
# 예제 6-8 가격 정보 가져오기
price = prod_items[0].select('li  p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price)

283880


In [12]:
# 예제 6-9 반복문으로 검색 결과의 1페이지에 대한 상품 정보 추출
prod_data = []
for prod_item in prod_items:
    try: # ① 상품명 가져오기
        title = prod_item.select('p.prod_name > a')[0].text.strip()
    except:
        title = ''
    try: # ② 스펙 목록 가져오기
        spec_list = prod_item.select('div.spec_list')[0].text.strip()
    except:
        spec_list = ''
    try: # ③ 가격 정보 가져오기 
        price = prod_item.select('li p.price_sect > a > strong')[0].text.strip().replace(",","")
    except:
        price = 0
    prod_data.append([title, spec_list, price])
print(len(prod_data))
print(prod_data[0])

30
['조지루시 EE-DVH35K', '가습기/가열식/수조, 분무구세척/[가습] 수조용량: 4.0L/연속사용: 10시간/최대분무량: 370cc/습도조절/습도표시/분무량조절/[편의] 상부급수식/타이머/[규격] 소비전력: 680W/크기(가로x세로x깊이): 240x365x275mm', '283880']


In [13]:
# 예제 6-10 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
         # ① 상품명 가져오기
        try:
            title = prod_item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ''
        # ② 스펙 목록 가져오기
        try: 
            spec_list = prod_item.select('div.spec_list')[0].text.strip()
        except:
            spec_list = ''
        # ③ 가격 정보 가져오기 
        try: 
            price = int(prod_item.select('li p.price_sect > a > strong')[0].text.strip().replace(",",""))
        except:
            price = 0
        prod_data.append([title, spec_list, price])
    return prod_data

In [14]:
# 예제 6-11 상품 정보를 가져오는 함수 테스트
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

30


# 6.2  데이터 수집 2 - 여러 페이지에 걸친 다나와 검색 페이지 크롤링

현재 다나와 검색 결과는 한 페이지당 30개씩 상품 정보를 보여주고 있다.

2 페이지를 선택했을 때의 변하는 URL 주소를 잘 확인해야 한다

### 6.2.1 다나와 검색 결과 페이지 URL 분석

In [15]:
# 예제 6-12 다나와 검색 URL을 만들어주는 함수 
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '가습기'
page = 1
url = get_search_page_url(keyword, page)
print(url)


http://search.danawa.com/dsearch.php?query=가습기&volumeType=allvs&page=1&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods


### 6.2.2 주피터 노트북의 진행표시줄 처리 

시간이 소요되는 진행을 확인하기 위해 표시한다.

In [58]:
# 예제 6-13 tqdm 라이브러리 설치 
#! pip install tqdm

In [59]:
# # 예제 6-14 tqdm 사용법 ===> 에러발생

'''
import time
from tqdm import tqdm_notebook
total_page = 10
for page in tqdm_notebook(range(1, total_page + 1)):
    # 페이지가 로딩 완료되기 위한 시간을 5초로 준다. 
    time.sleep(5)
'''
'''
import time
import tqdm
total_page = 10
for page in tqdm.notebook.tqdm(range(1, total_page + 1)):
    # 페이지가 로딩 완료되기 위한 시간을 5초로 준다. 
    time.sleep(5)
'''

'\nimport time\nimport tqdm\ntotal_page = 10\nfor page in tqdm.notebook.tqdm(range(1, total_page + 1)):\n    # 페이지가 로딩 완료되기 위한 시간을 5초로 준다. \n    time.sleep(5)\n'

### 6.2.3 여러 페이지에 걸친 상품 정보 수집

In [16]:
# 예제 6-15 실전 다나와 크롤링
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

# [selenium3] 에러발생 driver = webdriver.Chrome('D:/MyClass/Python/cWebConn/4_selenium/webdriver/chromedriver.exe')
# [selenium4]
driver = webdriver.Chrome(options=options)

# 암묵적으로 웹 자원 로드를 위해 3초까지 기다림
driver.implicitly_wait(3)
keyword = '가습기'
total_page = 10
prod_data_total = []
# 진행 정도를 표현하는 tqdm을 적용 ==> 오류발생
# for page in tqdm_notebook(range(1, total_page + 1)):
for page in range(1, total_page + 1):
    
    # ① 검색 페이지 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)
    # 페이지가 로딩 완료되기 위한 시간으로 5초를 할당
    time.sleep(5)
    
    # ② 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ③ 상품 정보 추출
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    # ④ 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list


### 6.2.4 수집 데이터 저장 

In [18]:
prod_data_total[0]

['조지루시 EE-DVH35K',
 '가습기/가열식/수조, 분무구세척/[가습] 수조용량: 4.0L/연속사용: 10시간/최대분무량: 370cc/습도조절/습도표시/분무량조절/[편의] 상부급수식/타이머/[규격] 소비전력: 680W/크기(가로x세로x깊이): 240x365x275mm',
 283880]

In [19]:
# 예제 6-16 데이터 저장
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('./files/3_1_danawa_crawling_result.xlsx', index = False)